## Problem Description

A current of I = 16 A is homogeneously distributed along the cross section of a cylindrical
copper wire with radius r1 = 2 mm and length ℓz = 300 mm, aligned with the z-direction. The
wire is enclosed within a cylindrical, non-conducting insulating shell with inner radius r1, outer
radius r2 = 3.5 mm and with permeability µ = 5µ0, where µ0 is the permeability of vacuum.
The outer surface of the insulation shell is considered as a perfect electric conductor.

In [ ]:
# -*- coding: utf-8 -*-
"""
2021/2022
.. sectionauthor:: spaeck-leigsnering, bundschuh, ruppert, d'angelo, de gersem
"""
import gmsh
import numpy as np
import sys
from scipy import sparse
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from dataclasses import dataclass

%matplotlib inline

## Analytic model

In [ ]:
# %% Settings
r_1 = 2e-3                # [m]: inner radius (wire)
r_2 = 3.5e-3              # [m]: outer radius (shell)
depth = 300e-3            # [m]: Depth of wire (lz)
model_name = "wire"       # str : model name

I = 16                    # [A]   : applied current
J_0 = I/(np.pi*r_1**2)    # [A/m] : current density in the wire
mu_0 = 4*np.pi*1e-7       # [H/m] : permeability of vacuum (and inside of wire)
mu_shell = 5*mu_0         # [H/m] : permeability of shell

In [ ]:
def H_phi(r):
    """
    Analytic solution of Magnetic Field
    
    Parameters
    ----------
    r : np.ndarray 
        radius in [m]

    Returns
    -------
    h_phi : np.ndarray
        Magnetic field strength in [A/m]:
    """
    def H_phi_i(r):
        return J_0/2*r
    def H_phi_a(r):
        return J_0/2*r_1**2/r
    condition = r<r_1
    return condition*H_phi_i(r)+(~condition)*H_phi_a(r)

In [ ]:
def A_z(r):
    """
    Analytic solution of magnetic vector potential
    
    Parameters
    ----------
    r : np.ndarray 
        radius in [m]

    Returns
    -------
    a_z : np.ndarray 
        Magnetic vector potential in [Tm]
    """
    def A_z_i(r):
        return -I/(2*np.pi)*(mu_0/2*(r**2-r_1**2)/r_1**2+mu_shell*np.log(r_1/r_2))
    def A_z_a(r):
        return -mu_shell*I/(2*np.pi)*np.log(r/r_2)
    condition = r<r_1
    return condition*A_z_i(r)+(~condition)*A_z_a(r)

In [ ]:
# Plotting
r = np.linspace(1e-5,r_2,200)
#plt.plot(r,H_phi(r),'r-')
plt.plot(r,A_z(r),'b-')

In [ ]:
# Analytic Energy and Inductance
W_magn_analytic = I**2*depth/(4*np.pi)*(mu_0/4+mu_shell*np.log(r_2/r_1)) # [J]   : magnetic energy (analytical)
L_analytic = 2*W_magn_analytic/I**2                                      # [H]   : inductance (analytical)

## Construct finite-element model
### 1) Import Mesh and physicals groups from Gmsh file

In [ ]:
# Handling of gmsh data in Python context. (Provide to students) 
# This implementation is well suited for small meshes. 
@dataclass 
class Mesh:
    node_tag_data : np.ndarray
    node_data : np.ndarray
    elementTypes : np.ndarray
    element_tags : np.ndarray
    nodeTags_elements : np.ndarray
    
    #treat nodes
    @property
    def num_node(self):
        # number of nodes
        return int(len(self.node_data)/3) 

    @property
    def node(self):
        # nodes
        node = np.reshape(self.node_data, (self.num_node, 3))
        # coordinates of nodes. x-coordinate in first column 
        # and y-coordinate in second column
        node = node[:, 0:2]
        return node
    
    @property
    def node_tag(self):
        # ID of nodes
        node_tag = self.node_tag_data - np.ones(len(self.node_tag_data))
        node_tag = node_tag.astype('int')
        np.put_along_axis(self.node,np.c_[node_tag,node_tag],self.node,axis=0)
        return node_tag 
    
    #treat elements
    @property
    def ind_elements(self):
        # index of elements
        return np.where(self.elementTypes == 2)[0]
    
    @property 
    def elements(self):
        # elements
        return np.array(self.nodeTags_elements[self.ind_elements[0]])
    
    @property 
    def num_elements(self):
        # number of elements
        return int(len(self.elements)/3) 
    
    @property 
    def elem_to_node(self): 
        # Associate elements (triangles) and their respective nodes. 
        # Connection between elements and nodes. 
        # Each line contains the indices of the contained nodes
        elem_to_node= np.reshape(self.elements, (self.num_elements, 3)) - np.ones(
            (self.num_elements, 1))
        elem_to_node = elem_to_node.astype('int')
        return elem_to_node
    
    @property 
    def all_edges(self):
        # Contains all edges
        return np.r_[self.elem_to_node[:, [0, 1]], 
                     self.elem_to_node[:, [1, 2]], 
                     self.elem_to_node[:, [0, 2]]]
    
    @property
    def edge_to_node(self):
        # Associate edges and their respective nodes. 
        return np.unique(np.sort(self.all_edges), axis=0)  

#### Import Gmsh file and process mesh

In [ ]:
# Import Gmsh File 
# Alternatively, the model can be contructed directly with the python interface directly.

# Initialize gmsh
gmsh.initialize(sys.argv[0])
#gmsh.open(model_name+".msh")
gmsh.model.add(model_name)
gmsh.merge(model_name+".geo")
gmsh.option.setNumber('Mesh.MeshSizeFactor', 0.8) # control grid size here.
gmsh.option.setNumber('Mesh.MshFileVersion', 2.2) # MATLAB compatible mesh file format
gmsh.model.occ.synchronize()
#gmsh.fltk.run()                                  # uncomment, if you want to inspect the geometry
gmsh.model.mesh.generate(dim=2) # 2D mesh
#gmsh.fltk.run()                                  # uncomment, if you want to inspect the mesh
#gmsh.write(model_name+".msh")                    # writes the mesh to a file

# Read mesh data
node_tag, node, _ = gmsh.model.mesh.getNodes() 
elementTypes, element_tags, nodeTags_elements = gmsh.model.mesh.getElements()

# Process mesh data
msh = Mesh(node_tag, node,elementTypes,element_tags,nodeTags_elements)

#### Import Gmsh Physical groups and process

In [ ]:
# Import Physical Groups (Provide to student)
pg = gmsh.model.getPhysicalGroups()

# Physical Groups 
physical_groups = dict()
# In every entry in physical_groups we define the following structure (dimension, name, indices of all nodes)
for group in pg:
    physical_groups[group[1]] = (group[0], gmsh.model.getPhysicalName(group[0], group[1]),
                                 gmsh.model.mesh.getNodesForPhysicalGroup(group[0], group[1])[0] - 1)
    
# Finalize gmsh
gmsh.finalize()

In [ ]:
def entity_in_physical_group(physical_group_data: dict, entity2node: np.ndarray, identifier):
    """
    Computes the indices of all entities that are in the physical group 
    specified by the identifier
    
    Parameters
    ----------
    physical_group_data : dict
        dict with physical groups. Key is the ID of the group and value is a tuple with (dimension, name, indices of all nodes)
    entity2node : np.ndarray
        (K,N) array with K entities and N nodes per entity
    identifier : int or str
        Identifier of the physical group. The ID or the name of the group are accepted

    Returns
    -------
    entity_in_physical_group : np.ndarray
        (M,) array. With M being the number of entities in the physical group
    """
    if type(identifier) is str:
        for p in physical_group_data.keys():
            if physical_group_data[p][1] == identifier:
                identifier = p

    out = -1 * np.ones(entity2node.shape[0], dtype=int)
    for k in range(entity2node.shape[0]):
        if np.isin(entity2node[k, :], physical_group_data[identifier][2]).all():
            out[k] = k

    return out[out != -1]

### 2) Visualize Mesh structures

In [ ]:
# %% Plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(azim=0, elev=90)

x = np.array(msh.node[:, 0], ndmin=1).T
y = np.array(msh.node[:, 1], ndmin=1).T
z = np.zeros_like(x)

ax.plot_trisurf(x, y, z)

In [ ]:
# Alternative plotting. Here the result is directly 2D. 

triang = Triangulation(x, y,msh.elem_to_node)
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax1.set_aspect('equal')
tpc = ax1.triplot(triang)

# And here with z values
z = (np.cos(x) * np.cos(3 * y)).flatten()
ax2 = fig.add_subplot(122)
ax2.set_aspect('equal')
tpc = ax2.tripcolor(triang, z, shading='flat')
fig.colorbar(tpc)

In [ ]:
## Visualize the regions of the model 
# Indices of elements in shell and wire
elem_in_shell = entity_in_physical_group(physical_groups, msh.elem_to_node, 'SHELL')
elem_in_wire = entity_in_physical_group(physical_groups, msh.elem_to_node, 'WIRE')

# Indices of edges on ground
edges_on_ground = entity_in_physical_group(physical_groups, msh.edge_to_node, 'GND')

# Instead of using the name of the physical group, one can also use its ID:
#elem_in_shell = entity_in_physical_group(physical_groups, msh.elem_to_node, 2)
#elem_in_wire = entity_in_physical_group(physical_groups, msh.elem_to_node, 1)
#edges_on_ground = entity_in_physical_group(physical_groups, edge_to_node, 3)

triang_shell = Triangulation(x, y, msh.elem_to_node[elem_in_shell])
triang_wire = Triangulation(x, y, msh.elem_to_node[elem_in_wire])

# Plot shell, wire and edges on ground potential
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.triplot(triang_wire,color='orange',label='wire')
ax.triplot(triang_shell,color='blue',label='shell')
plt.legend()

for edge in edges_on_ground:
    node_x = msh.node[msh.edge_to_node[edge, [0, 1]], 0]
    node_y = msh.node[msh.edge_to_node[edge, [0, 1]], 1]
    line, = ax.plot(node_x, node_y, color='red')
line.set_label('ground')

### 3) Define the finite-element shape functions, construct stiffness matrix and load vector 

In [ ]:
# Task 4: Calculate the element areas (element_area, numelem-by-1) and the 
# further coefficients a, b and c (each numelem-by-3)
# Define the finite-element shape functions 

# Definition of shape function N = a+bx+cy
@dataclass 
class ShapeFunction_N:
    depth : float
    element_area : np.ndarray
    a : np.ndarray
    b : np.ndarray
    c : np.ndarray
    
x1 = msh.node[msh.elem_to_node, 0]  # x coordinates of the nodes for each element
y1 = msh.node[msh.elem_to_node, 1]  # y coordinates of the nodes for each element
x2 = np.roll(x1, -1, axis=1)
y2 = np.roll(y1, -1, axis=1)
x3 = np.roll(x1, -2, axis=1)
y3 = np.roll(y1, -2, axis=1)


# Definition of coefficients according to HDG's lecture. Alternative approaches exist 
# and are equivalent.
a = x2*y3-x3*y2
b = y2-y3
c = x3-x2

element_area = np.mean(((x2*y3-x3*y2)+(y2-y3)*x1+(x3-x2)*y1)/2,1)

# Instantiation 
shape_function = ShapeFunction_N(depth,element_area,a,b,c)

In [ ]:
# Compute element-wise reluctivity
reluctivity_in_elements = 1/mu_0 * np.ones(msh.num_elements) # [m/H] : reluctivities, one per element
reluctivity_in_elements[elem_in_shell] = 1/mu_shell # [m/H] : reluctivities for the iron shell

# Plotting 
fig = plt.figure()
ax = fig.add_subplot(111)
tpc = ax.tripcolor(triang, facecolors=reluctivity_in_elements)
fig.colorbar(tpc)
ax.triplot(triang, color='black', lw=0.1)
ax.set_aspect('equal', 'box')

In [ ]:
# Task 4: Construct the stiffness matrix Knu (H−1). Do this at first by looping over all elements. 
# Gather the row numbers, column numbers and matrix entries into vectors i, j, v. 
# At the end, use the class sparse to assemble all entries into a sparse matrix Knu. 
# Show the sparsity pattern of Knu using the function spy


# Define rotrot Operator for the 2D longitudinal case --> gradient!
# for each element k
def Knu_for_elem(k):
    """Local matrix"""
    grad_Ni_grad_Nj = (np.array(shape_function.b[k, :], ndmin=2).T @ np.array(shape_function.b[k, :], ndmin=2) +
           np.array(shape_function.c[k, :], ndmin=2).T @ np.array(shape_function.c[k, :], ndmin=2)) \
        / (4*shape_function.element_area[k]*shape_function.depth)
    return reluctivity_in_elements[k] * grad_Ni_grad_Nj

# Assign Knu for global indices
index_rows = np.zeros((9*msh.num_elements), dtype='int') 
index_columns = np.zeros((9*msh.num_elements), dtype='int')
elementwise_entries = np.zeros((9*msh.num_elements)) 
for k in range(msh.num_elements):
    global_indices = msh.elem_to_node[k, :] # get global indices for element k
    triple_global_indices = np.array([global_indices, global_indices, global_indices])
    index_rows[9*k:9*k+9] = np.reshape(triple_global_indices, (9))  # update global row vector
    index_columns[9*k:9*k+9] = np.reshape(triple_global_indices.T, (9))  # update global column vector
    elementwise_entries[9*k:9*k+9] = np.reshape(Knu_for_elem(k), (9))     # global elementwise entry vector

# Assembly of Knu
index_rows = index_rows.T
index_columns = index_columns.tolist()
elementwise_entries = elementwise_entries.tolist()
# Build Knu Matrix
Knu = sparse.csr_matrix((elementwise_entries, (index_rows, index_columns))) # [1/H] : circuit-reluctance matrix

# Print structure of matrix
plt.spy(Knu, markersize=1)

In [ ]:
# Task 4: Define load vector j_grid, element-wise current density

# Set element-wise current density in wire region
current_density_in_elems = np.zeros(msh.num_elements) # [A/m^2]: current density
current_density_in_elems[elem_in_wire] = I/np.sum(shape_function.element_area[elem_in_wire]) # [A/m^2]: current density
grid_currents = current_density_in_elems*shape_function.element_area/3 # localized current density, grid current [A]
grid_currents = np.tile(grid_currents, (3, 1)).transpose() # inflate for nodal accumulation

values = np.zeros(msh.num_node)
for k in range(0, 3):
    for i in range(0, msh.num_elements-1):
        idx_node = msh.elem_to_node[i,k]
        values[idx_node] += grid_currents[i,k]

# Assembly of grid current vector
j_grid = sparse.csr_matrix((values, (np.arange(msh.num_node), 
                                     np.zeros(msh.num_node))), shape=(msh.num_node, 1))


# Plotting 
fig = plt.figure()
ax = fig.add_subplot(111)
tpc = ax.tripcolor(triang, facecolors=current_density_in_elems)
fig.colorbar(tpc)
ax.triplot(triang, color='black', lw=0.1)
ax.set_aspect('equal', 'box')

In [ ]:
# Task 5: First validation is the check of the magnetic energy
r = np.sqrt(x**2+y**2)                         # [m]    : radial coordinate
A_analytic = depth*A_z(r)                      # [Tm]   : analytic solution for the magnetic vector potential projected onto the mesh
W_magn_test = 1/2*A_analytic@Knu@A_analytic    # [J]    : magnetic energy (analytic solution, numerical circuit-reluctance matrix)

print('Magnetic energy (analytic solution)           :',W_magn_analytic,'J')
print('Magnetic energy (analytic Az, numerical Knu)  :',W_magn_test,'J')

### 4) Enter boundary conditions and solve the system

In [ ]:
a = np.zeros((msh.num_node, 1))     # Initialize vector of dofs

# Indices and values of nodes at Dirichlet boundary
index_constraint = physical_groups[3] # [@]: edge indices of the shell boundary on GND
index_constraint = index_constraint[2] # strip indices
value_constraint = np.zeros((len(index_constraint),1)) # [@]: indices of the nodes affected by homogeneous boundary condition
index_dof = np.setdiff1d(np.arange(msh.num_node), index_constraint).tolist()  # [@]: indices of the degrees of freedom

# Shrink the system
Knu_shrink = Knu[index_dof, :] # remove boundary condition (=known) entries in Knu matrix
Knu_shrink = Knu_shrink[:, index_dof] 
j_grid_dof = j_grid[index_dof] 
rhs = j_grid_dof
# Solve the system
a_shrink = spsolve(Knu_shrink, rhs)

# Inflate
a_shrink = np.array(a_shrink, ndmin=2).T
a[index_dof] = a_shrink
a[index_constraint] = value_constraint
a = a.reshape(len(a))

# Visualization 
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal', 'box')

surf = ax.tripcolor(triang, a, cmap='viridis')  # cmap=plt.cm.CMRmap)
fig.colorbar(surf, shrink=0.5, aspect=5)

### 5) Post Process

In [ ]:
#Calculate magnetic flux density
b_field = np.vstack([np.sum(shape_function.c*a[msh.elem_to_node[:]]
                    /(2*shape_function.element_area[:,None]),1)
                    /shape_function.depth,
                    np.sum(shape_function.b*a[msh.elem_to_node[:]]
                    /(2*shape_function.element_area[:,None]),1)
                    /shape_function.depth]).T
b_field_abs = np.linalg.norm(b_field, axis=1)    # [T]    : magnitude of the magnetic flux density

# Plotting 
fig = plt.figure()
ax = fig.add_subplot(111)
tpc = ax.tripcolor(triang, facecolors=b_field_abs)
fig.colorbar(tpc)
ax.triplot(triang, color='black', lw=0.1)
ax.set_aspect('equal', 'box')

# Validity check
W_magn_fe = 1/2*a@Knu@a                                                 # [J]    : magnetic energy
W_magn_fe2 = np.sum(1/2*reluctivity_in_elements*b_field_abs**2
                    *shape_function.element_area*shape_function.depth)  # [J]     : magnetic energy (integrated)
print('Validity Check:')
print('Magnetic energy (analytic solution)                :',W_magn_analytic,'J')
print('Magnetic energy (FE solution)                      :',W_magn_fe,'J')
print('Magnetic energy (numerical solution, integrated)   :',W_magn_fe2,'J')

## Convergence Study

In [ ]:
"""
We automate the FE procedure such that the calculations are repeated for models 
with different mesh sizes, created by different values for the mesh size option.
"""

refinements_steps = [0, 1, 2, 3] # variable for the mesh size refinement
energies = np.empty_like(refinements_steps, dtype=float) # outputs for the convergence plot
element_sizes = np.empty_like(refinements_steps, dtype=float)

"""Iterate over refinement steps. In every step:
- Solve the Problem
- Compute the energy
- Determine the maximum element size
"""
#to be continued...